In [10]:
# ONLY RUN ONCE, must restart notebook otherwise
import pandas as pd
import os
from pathlib import Path
current_dir = Path.cwd()
path = current_dir.parent
print(path)
#change directory to get src class data
os.chdir(path)
from src.data_object import *
from src.transforms import *

from src.calculable_object import *
from src.wellbore_trajectory import *

C:\Users\BpAmos\Documents\repos


In [11]:
data_path = 'C:/Users/BpAmos/Dropbox (Personal)/Consulting/QRI/SDX/DAT/ABKATUN'

In [12]:
dev_files = get_files(data_path, folders='Deviations', extensions='.dev')
dev_files.items[0].stem

'Abkatun0001a'

In [13]:

appended_df = pd.DataFrame()
for item in dev_files.items:
    
    df = pd.read_csv(item, header=None, skiprows=15, sep = ' ')

    df.drop(columns=[0], inplace=True)

    header = list(df.iloc[0])

    header =  [x for x in header if str(x) != 'nan']


    df = df.iloc[2:]

    df.dropna(how='all', axis=1, inplace=True)

    df.columns = header
    df['wellId'] = item.stem
    df.reset_index(drop=True,inplace = True)
    
    df['MD'] = df['MD']*3.28084
    
    df['XSurf'] = df.iloc[0].X
    df['YSurf'] = df.iloc[0].Y
    
    appended_df = appended_df.append(df)
    
appended_df

,MD,X,Y,Z,TVD,DX,DY,AZIM,INCL,DLS,wellId,XSurf,YSurf
0,0.00000,587120.00,2133594.00,30.00,0.00,0.0000000000,0.00,0.000000,0.000000,0.0,Abkatun0001a,587120.0,2133594.0
1,11820.86652,587120.00,2133594.00,-3573.00,3603.00,0.0000000000,0.00,0.000000,0.000000,0.0,Abkatun0001a,587120.0,2133594.0
0,0.00000,587117.00,2133591.00,30.00,0.00,0.0000000000,0.00,0.000000,0.000000,0.0,Abkatun0001b,587117.0,2133591.0
1,11102.36256,587117.00,2133591.00,-3354.00,3384.00,0.0000000000,0.00,0.000000,0.000000,0.0,Abkatun0001b,587117.0,2133591.0
0,0.00000,585593.00,2131138.00,26.00,0.00,0.0000000000,0.00,136.847610,0.125659,0.0,Abkatun0004,585593.0,2131138.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,14304.46240,581682.10,2116903.63,-4204.33,4234.33,-727.9000000,-337.37,266.000770,12.000263,0.0,Abkatun2141,582410.0,2117241.0
79,14468.50440,581671.73,2116902.91,-4253.24,4283.24,-738.2700000,-338.09,266.002698,12.004661,0.0,Abkatun2141,582410.0,2117241.0
80,14632.54640,581661.35,2116902.18,-4302.15,4332.15,-748.6500000,-338.82,266.002703,12.005852,0.0,Abkatun2141,582410.0,2117241.0
81,14796.58840,581650.98,2116901.46,-4351.05,4381.05,-759.0200000,-339.54,265.868016,11.963039,0.0,Abkatun2141,582410.0,2117241.0


In [14]:
appended_df = appended_df[['MD','INCL','AZIM','wellId','XSurf','YSurf']]
#appended_df

In [15]:
grouped = appended_df.groupby('wellId')
data = ""
for name, group in grouped:
    #print(name)
    #print(group)
    data = group
group
group.reset_index(inplace=True, drop=True)
group

,MD,INCL,AZIM,wellId,XSurf,YSurf
0,0.00000,0.000000,0.000000,Abkatun2141,582410.0,2117241.0
1,164.04200,0.000000,0.000000,Abkatun2141,582410.0,2117241.0
2,328.08400,0.000000,0.000000,Abkatun2141,582410.0,2117241.0
3,492.12600,0.000000,0.000000,Abkatun2141,582410.0,2117241.0
4,656.16800,0.000000,0.000000,Abkatun2141,582410.0,2117241.0
...,...,...,...,...,...,...
78,14304.46240,12.000263,266.000770,Abkatun2141,582410.0,2117241.0
79,14468.50440,12.004661,266.002698,Abkatun2141,582410.0,2117241.0
80,14632.54640,12.005852,266.002703,Abkatun2141,582410.0,2117241.0
81,14796.58840,11.963039,265.868016,Abkatun2141,582410.0,2117241.0


In [16]:
def from_df(df, wellId_name: Optional[str] = None, md_name: Optional[str] = None,
            inc_name: Optional[str] = None, azim_name: Optional[str] = None,
            surface_latitude_name: Optional[str] = None,
            surface_longitude_name: Optional[str] = None,
            surface_x_name: Optional[str] = None,
            surface_y_name: Optional[str] = None):
    
        # if no column names are specified use 0:5 for dir survey obj
    wellId_name = if_none(wellId_name, df.columns[0])
    md_name = if_none(md_name, df.columns[1])
    inc_name = if_none(inc_name, df.columns[2])
    azim_name = if_none(azim_name, df.columns[3])
    surface_latitude_name = if_none(surface_latitude_name, None)
    surface_longitude_name = if_none(surface_longitude_name, None)
    surface_x_name = if_none(surface_x_name, None)
    surface_y_name = if_none(surface_y_name, None)
    
    
    if surface_latitude_name is not None and surface_y_name is not None:
        dataclass_obj = dict(wellId=df[wellId_name][0],
                             md=np.array(df[md_name]),
                             inc=np.array(df[inc_name]),
                             azim=np.array(df[azim_name]),
                             surface_latitude=df[surface_latitude_name][0],
                             surface_longitude=df[surface_longitude_name][0])
        
    if surface_x_name is not None and surface_y_name is not None:
        dataclass_obj = dict(wellId=df[wellId_name][0],
                             md=np.array(df[md_name]),
                             inc=np.array(df[inc_name]),
                             azim=np.array(df[azim_name]),
                             surface_x=df[surface_x_name][0],
                             surface_y=df[surface_y_name][0])
        
    return dataclass_obj

In [17]:
my_well_dict = from_df(group, wellId_name='wellId', md_name='MD', inc_name='INCL', azim_name='AZIM',
                       surface_x_name ='XSurf', surface_y_name = 'YSurf')

#my_well_dict

In [21]:
well_obj = WellboreTrajectory(my_well_dict)
well_obj.calculate_survey_points(crs_in = 'EPSG:2027', horizontal_angle=20.0)
well_obj.deviation_survey_obj

crs_in EPSG:2027
horizontal_angle 20.0
EPSG:2027
YES


DeviationSurvey(wellId='Abkatun2141', md=array([    0.     ,   164.042  ,   328.084  ,   492.126  ,   656.168  ,
         820.21   ,   984.252  ,  1148.294  ,  1312.336  ,  1476.378  ,
        1640.42   ,  1804.462  ,  1968.504  ,  2132.546  ,  2296.588  ,
        2460.63   ,  2624.672  ,  2788.714  ,  2952.756  ,  3116.798  ,
        3280.84   ,  3444.882  ,  3608.924  ,  3772.966  ,  3897.63792,
        4192.91352,  4222.44108,  4337.27048,  4458.66156,  4583.33348,
        4708.0054 ,  4832.67732,  4954.0684 ,  5013.12352,  5154.19964,
        5370.73508,  5505.24952,  5702.09992,  5839.8952 ,  5994.09468,
        6338.58288,  6482.93984,  6656.82436,  6774.9346 ,  6860.23644,
        6998.03172,  7099.73776,  7191.60128,  7349.0816 ,  7500.00024,
        7611.5488 ,  7769.02912,  7923.2286 ,  8169.2916 ,  8303.80604,
        8425.19712,  8576.11576,  8753.28112,  8884.51472,  9068.24176,
        9251.9688 ,  9435.69584,  9619.42288,  9799.86908, 10009.84284,
       10351.0502 , 105

In [ ]:
my_df = well_obj.get_survey_df()
my_df

In [ ]:
exportPath = 'C:/Users/BpAmos/Desktop/TMP/abkatunOneWell.csv'
my_df.to_csv(os.path.join(exportPath),index=False)

In [24]:
grouped = merge.groupby('wellId')

d={}
dlist=[]
for name, group in grouped:
    print(name)
    #print(group)
    group.reset_index(inplace=True, drop=True)
    #print(group)
    my_well_dict = from_df(group, wellId_name='wellId', md_name='MD', inc_name='INCL', azim_name='AZIM',
                           surface_latitude_name ='latitude', surface_longitude_name = 'longitude')
    #print(my_well_dict)
    well_obj = WellboreTrajectory(my_well_dict)
    well_obj.calculate_survey_points()
    well_obj.deviation_survey_obj
    #print(well_obj.deviation_survey_obj)
    
    dataclass_obj = {'wellId': well_obj.deviation_survey_obj.wellId,
                                  'md': well_obj.deviation_survey_obj.md,
                                  'inc': well_obj.deviation_survey_obj.inc,
                                  'azim': well_obj.deviation_survey_obj.azim,
                                  'tvd': well_obj.deviation_survey_obj.tvd,
                                  'e_w_deviation': well_obj.deviation_survey_obj.e_w_deviation,
                                  'n_s_deviation': well_obj.deviation_survey_obj.n_s_deviation,
                                  'dls': well_obj.deviation_survey_obj.dls,
                                  'surface_latitude': well_obj.deviation_survey_obj.surface_latitude,
                                  'surface_longitude': well_obj.deviation_survey_obj.surface_longitude,
                                  'longitude_points': well_obj.deviation_survey_obj.longitude_points,
                                  'latitude_points': well_obj.deviation_survey_obj.latitude_points,
                                  'zone_number': well_obj.deviation_survey_obj.zone_number,
                                  'zone_letter': well_obj.deviation_survey_obj.zone_letter,
                                  'x_points': well_obj.deviation_survey_obj.x_points,
                                  'y_points': well_obj.deviation_survey_obj.y_points,
                                  'surface_x': well_obj.deviation_survey_obj.surface_x,
                                  'surface_y': well_obj.deviation_survey_obj.surface_y,
                                  'isHorizontal': well_obj.deviation_survey_obj.isHorizontal
                                  }
    
    
    d=dataclass_obj
    print(d)
    dlist.append(d)
    
#dlist

Abkatun0001a
{'wellId': 'Abkatun0001a', 'md': array([    0.     , 11820.86652]), 'inc': array([0., 0.]), 'azim': array([0., 0.]), 'tvd': array([    0.     , 11820.86652]), 'e_w_deviation': array([0., 0.]), 'n_s_deviation': array([0., 0.]), 'dls': array([0., 0.]), 'surface_latitude': 19.29539896465997, 'surface_longitude': -92.17081663030581, 'longitude_points': array([-92.17081651, -92.17081651]), 'latitude_points': array([19.29539896, 19.29539896]), 'zone_number': 15, 'zone_letter': 'Q', 'x_points': array([587118.69833896, 587118.69833896]), 'y_points': array([2133721.53442718, 2133721.53442718]), 'surface_x': 587118.6983389647, 'surface_y': 2133721.5344271804, 'isHorizontal': array(['Vertical', 'Vertical'], dtype='<U10')}
Abkatun0001b
{'wellId': 'Abkatun0001b', 'md': array([    0.     , 11102.36256]), 'inc': array([0., 0.]), 'azim': array([0., 0.]), 'tvd': array([    0.     , 11102.36256]), 'e_w_deviation': array([0., 0.]), 'n_s_deviation': array([0., 0.]), 'dls': array([0., 0.]), 's

{'wellId': 'Abkatun0021', 'md': array([    0.     ,    98.4252 ,   196.8504 ,   295.2756 ,   393.7008 ,
         492.126  ,   590.5512 ,   688.9764 ,   787.4016 ,   885.8268 ,
         984.252  ,  1082.6772 ,  1181.1024 ,  1279.5276 ,  1377.9528 ,
        1476.378  ,  1574.8032 ,  1673.2284 ,  1771.6536 ,  1870.0788 ,
        1968.504  ,  2066.9292 ,  2165.3544 ,  2263.7796 ,  2362.2048 ,
        2460.63   ,  2559.0552 ,  2657.4804 ,  2755.9056 ,  2854.3308 ,
        2952.756  ,  3051.1812 ,  3149.6064 ,  3248.0316 ,  3346.4568 ,
        3444.882  ,  3543.3072 ,  3641.7324 ,  3740.1576 ,  3838.5828 ,
        3937.008  ,  4035.4332 ,  4133.8584 ,  4232.2836 ,  4330.7088 ,
        4429.134  ,  4527.5592 ,  4625.9844 ,  4724.4096 ,  4822.8348 ,
        4921.26   ,  5019.6852 ,  5118.1104 ,  5216.5356 ,  5314.9608 ,
        5413.386  ,  5511.8112 ,  5610.2364 ,  5708.6616 ,  5807.0868 ,
        5905.512  ,  6003.9372 ,  6102.3624 ,  6200.7876 ,  6299.2128 ,
        6397.638  ,  6496.0632 ,

{'wellId': 'Abkatun0031', 'md': array([    0.     ,    98.4252 ,   196.8504 ,   295.2756 ,   393.7008 ,
         492.126  ,   590.5512 ,   688.9764 ,   787.4016 ,   885.8268 ,
         984.252  ,  1082.6772 ,  1181.1024 ,  1279.5276 ,  1377.9528 ,
        1476.378  ,  1574.8032 ,  1673.2284 ,  1771.6536 ,  1870.0788 ,
        1968.504  ,  2066.9292 ,  2165.3544 ,  2326.11556,  2414.69824,
        2513.12344,  2601.70612,  2696.85048,  2788.714  ,  2910.10508,
        3005.24944,  3083.9896 ,  3208.66152,  3300.52504,  3395.6694 ,
        3487.53292,  3599.08148,  3690.945  ,  3779.52768,  3900.91876,
        3969.8164 ,  4094.48832,  4150.2626 ,  4212.59856,  4337.27048,
        4491.46996,  4645.66944,  4704.72456,  4776.90304,  4908.13664,
        5036.0894 ,  5134.5146 ,  5232.9398 ,  5321.52248,  5410.10516,
        5501.96868,  5593.8322 ,  5688.97656,  5780.84008,  5872.7036 ,
        5958.00544,  6082.67736,  6200.7876 ,  6295.93196,  6414.0422 ,
        6781.49628,  7158.79288,

{'wellId': 'Abkatun0057', 'md': array([    0.     ,    98.4252 ,   196.8504 ,   295.2756 ,   393.7008 ,
         492.126  ,   590.5512 ,   688.9764 ,   787.4016 ,   885.8268 ,
         984.252  ,  1082.6772 ,  1181.1024 ,  1279.5276 ,  1377.9528 ,
        1476.378  ,  1574.8032 ,  1673.2284 ,  1771.6536 ,  1870.0788 ,
        1968.504  ,  2066.9292 ,  2165.3544 ,  2263.7796 ,  2362.2048 ,
        2460.63   ,  2559.0552 ,  2657.4804 ,  2755.9056 ,  2854.3308 ,
        2952.756  ,  3051.1812 ,  3149.6064 ,  3248.0316 ,  3346.4568 ,
        3444.882  ,  3543.3072 ,  3641.7324 ,  3740.1576 ,  3838.5828 ,
        3937.008  ,  4035.4332 ,  4133.8584 ,  4232.2836 ,  4330.7088 ,
        4429.134  ,  4527.5592 ,  4625.9844 ,  4724.4096 ,  4822.8348 ,
        4921.26   ,  5019.6852 ,  5118.1104 ,  5216.5356 ,  5314.9608 ,
        5413.386  ,  5511.8112 ,  5610.2364 ,  5708.6616 ,  5807.0868 ,
        5905.512  ,  6003.9372 ,  6102.3624 ,  6200.7876 ,  6299.2128 ,
        6594.4884 ,  6709.3178 ,

{'wellId': 'Abkatun0059', 'md': array([    0.     ,    98.4252 ,   196.8504 ,   295.2756 ,   393.7008 ,
         492.126  ,   590.5512 ,   688.9764 ,   787.4016 ,   885.8268 ,
         984.252  ,  1082.6772 ,  1181.1024 ,  1279.5276 ,  1377.9528 ,
        1476.378  ,  1574.8032 ,  1673.2284 ,  1771.6536 ,  1804.462  ,
        1870.0788 ,  1968.504  ,  2066.9292 ,  2165.3544 ,  2263.7796 ,
        2362.2048 ,  2460.63   ,  2559.0552 ,  2657.4804 ,  2755.9056 ,
        2854.3308 ,  2952.756  ,  3051.1812 ,  3149.6064 ,  3248.0316 ,
        3346.4568 ,  3444.882  ,  3543.3072 ,  3641.7324 ,  3740.1576 ,
        3838.5828 ,  3937.008  ,  4035.4332 ,  4133.8584 ,  4232.2836 ,
        4330.7088 ,  4429.134  ,  4527.5592 ,  4625.9844 ,  4724.4096 ,
        4822.8348 ,  4921.26   ,  5019.6852 ,  5118.1104 ,  5216.5356 ,
        5314.9608 ,  5413.386  ,  5511.8112 ,  5518.37288,  5610.2364 ,
        5659.449  ,  5708.6616 ,  5807.0868 ,  5905.512  ,  6003.9372 ,
        6102.3624 ,  6200.7876 ,

...

{'wellId': 'Abkatun0216', 'md': array([    0.     ,  2296.588  ,  2375.32816,  2437.66412,  2660.76124,
        2693.56964,  2755.9056 ,  2847.76912,  2939.63264,  3028.21532,
        3211.94236,  3339.89512,  3523.62216,  3651.57492,  3805.7744 ,
        3992.78228,  4117.4542 ,  4311.02376,  4520.99752,  4698.16288,
        4881.88992,  5075.45948,  5216.5356 ,  5298.5566 ,  5603.67472,
        6627.2968 ,  7014.43592,  7385.17084,  7729.65904,  8070.8664 ,
        8536.74568,  8792.6512 ,  9012.46748,  9402.88744, 10370.73524,
       11092.52004, 11345.14472, 11496.06336, 11722.44132, 12985.56472]), 'inc': array([ 0.        ,  1.57107076,  3.50672224,  6.22601889, 15.61340412,
       17.32267642, 20.68893386, 24.24691013, 26.26344162, 27.80918919,
       30.08695715, 29.8794384 , 28.2609402 , 27.73962931, 27.73914212,
       27.7379229 , 27.84679027, 28.05907165, 28.12417691, 28.30869062,
       28.74351699, 29.07075086, 29.11972164, 29.0965897 , 29.3407942 ,
       31.57462876, 31.

      dtype='<U10')}
Abkatun0223a
{'wellId': 'Abkatun0223a', 'md': array([    0.       ,    98.4252   ,   196.8504   ,   295.2756   ,
         393.7008   ,   492.126    ,   590.5512   ,   688.9764   ,
         787.4016   ,   885.8268   ,   984.252    ,  1082.6772   ,
        1181.1024   ,  1279.5276   ,  1377.9528   ,  1476.378    ,
        1574.8032   ,  1673.2284   ,  1771.6536   ,  1870.0788   ,
        1968.504    ,  2066.9292   ,  2165.3544   ,  2263.7796   ,
        2362.2048   ,  2460.63     ,  2559.0552   ,  2657.4804   ,
        2755.9056   ,  2854.3308   ,  2952.756    ,  3051.1812   ,
        3149.6064   ,  3248.0316   ,  3346.4568   ,  3444.882    ,
        3543.3072   ,  3641.7324   ,  3740.1576   ,  3838.5828   ,
        3937.008    ,  4035.4332   ,  4133.8584   ,  4232.2836   ,
        4330.7088   ,  4429.134    ,  4527.5592   ,  4625.9844   ,
        4724.4096   ,  4822.8348   ,  4921.26     ,  5019.6852   ,
        5118.1104   ,  5216.5356   ,  5314.9608   ,  5413.386 

{'wellId': 'Abkatun2013', 'md': array([    0.      ,    98.4252  ,   196.8504  ,   295.2756  ,
         393.7008  ,   492.126   ,   590.5512  ,   688.9764  ,
         787.4016  ,   885.8268  ,   984.252   ,  1082.6772  ,
        1181.1024  ,  1279.5276  ,  1377.9528  ,  1476.378   ,
        1574.8032  ,  1673.2284  ,  1771.6536  ,  1870.0788  ,
        1968.504   ,  2066.9292  ,  2165.3544  ,  2263.7796  ,
        2362.2048  ,  2460.63    ,  2559.0552  ,  2657.4804  ,
        2755.9056  ,  2854.3308  ,  2952.756   ,  3051.1812  ,
        3149.6064  ,  3248.0316  ,  3346.4568  ,  3444.882   ,
        3543.3072  ,  3641.7324  ,  3740.1576  ,  3838.5828  ,
        3937.008   ,  4035.4332  ,  4133.8584  ,  4232.2836  ,
        4330.7088  ,  4429.134   ,  4527.5592  ,  4625.9844  ,
        4724.4096  ,  4822.8348  ,  4921.26    ,  5019.6852  ,
        5118.1104  ,  5216.5356  ,  5314.9608  ,  5413.386   ,
        5511.8112  ,  5610.2364  ,  5708.6616  ,  5807.0868  ,
        5905.512   ,  6

{'wellId': 'Abkatun2041', 'md': array([    0.       ,    98.4252   ,   196.8504   ,   295.2756   ,
         393.7008   ,   492.126    ,   590.5512   ,   688.9764   ,
         787.4016   ,   885.8268   ,   984.252    ,  1082.6772   ,
        1181.1024   ,  1279.5276   ,  1377.9528   ,  1476.378    ,
        1574.8032   ,  1673.2284   ,  1771.6536   ,  1870.0788   ,
        1968.504    ,  2066.9292   ,  2165.3544   ,  2263.7796   ,
        2362.2048   ,  2460.63     ,  2559.0552   ,  2657.4804   ,
        2755.9056   ,  2854.3308   ,  2952.756    ,  3051.1812   ,
        3149.6064   ,  3248.0316   ,  3346.4568   ,  3444.882    ,
        3543.3072   ,  3641.7324   ,  3740.1576   ,  3838.5828   ,
        3937.008    ,  4035.4332   ,  4133.8584   ,  4232.2836   ,
        4330.7088   ,  4429.134    ,  4527.5592   ,  4625.9844   ,
        4724.4096   ,  4822.8348   ,  4921.26     ,  5019.6852   ,
        5118.1104   ,  5216.5356   ,  5314.9608   ,  5413.386    ,
        5511.8112   ,  5610.23

      dtype='<U10')}
Abkatun2121
{'wellId': 'Abkatun2121', 'md': array([    0.      ,    98.4252  ,   196.8504  ,   295.2756  ,
         393.7008  ,   492.126   ,   590.5512  ,   688.9764  ,
         787.4016  ,   885.8268  ,   984.252   ,  1082.6772  ,
        1181.1024  ,  1279.5276  ,  1377.9528  ,  1476.378   ,
        1574.8032  ,  1673.2284  ,  1771.6536  ,  1870.0788  ,
        1968.504   ,  2066.9292  ,  2165.3544  ,  2263.7796  ,
        2362.2048  ,  2460.63    ,  2540.026328,  2632.217932,
        2723.753368,  2816.60114 ,  2907.808492,  3001.312432,
        3091.863616,  3183.070968,  3363.845252,  3455.708772,
        3549.540796,  3641.404316,  3733.267836,  3823.81902 ,
        3916.666792,  4008.858396,  4100.721916,  4193.241604,
        4286.089376,  4378.609064,  4470.800668,  4563.320356,
        4656.496212,  4749.0159  ,  4842.191756,  4935.367612,
        5027.559216,  5120.078904,  5209.97392 ,  5300.19702 ,
        5438.320384,  5482.939808,  5576.115664,  566

In [25]:
df_final = pd.DataFrame()
for i in dlist:
    df1 = pd.DataFrame(i)
    #print(df1)
    df_final = df_final.append(df1)
df_final

,wellId,md,inc,azim,tvd,e_w_deviation,n_s_deviation,dls,surface_latitude,surface_longitude,longitude_points,latitude_points,zone_number,zone_letter,x_points,y_points,surface_x,surface_y,isHorizontal
0,Abkatun0001a,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.295399,-92.170817,-92.170817,19.295399,15,Q,587118.698339,2.133722e+06,587118.698339,2.133722e+06,Vertical
1,Abkatun0001a,11820.86652,0.000000,0.000000,11820.866520,0.000000,0.000000,0.000000,19.295399,-92.170817,-92.170817,19.295399,15,Q,587118.698339,2.133722e+06,587118.698339,2.133722e+06,Vertical
0,Abkatun0001b,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.295372,-92.170845,-92.170845,19.295372,15,Q,587115.698385,2.133719e+06,587115.698385,2.133719e+06,Vertical
1,Abkatun0001b,11102.36256,0.000000,0.000000,11102.362560,0.000000,0.000000,0.000000,19.295372,-92.170845,-92.170845,19.295372,15,Q,587115.698385,2.133719e+06,587115.698385,2.133719e+06,Vertical
0,Abkatun0004,0.00000,0.125659,136.847610,0.000000,0.000000,0.000000,0.000000,19.273661,-92.185968,-92.185967,19.273661,15,Q,585538.062715,2.131308e+06,585538.062715,2.131308e+06,Vertical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Abkatun2141,14304.46240,12.000263,266.000770,13893.159542,-2391.663258,-1112.140355,0.000000,19.144775,-92.223586,-92.230532,19.141741,15,Q,580919.546093,2.116689e+06,581648.525054,2.117028e+06,Vertical
79,Abkatun2141,14468.50440,12.004661,266.002698,14053.615361,-2425.693395,-1114.518871,0.002681,19.144775,-92.223586,-92.230630,19.141735,15,Q,580909.173707,2.116688e+06,581648.525054,2.117028e+06,Vertical
80,Abkatun2141,14632.54640,12.005852,266.002703,14214.069517,-2459.731380,-1116.897359,0.000726,19.144775,-92.223586,-92.230729,19.141729,15,Q,580898.798929,2.116688e+06,581648.525054,2.117028e+06,Vertical
81,Abkatun2141,14796.58840,11.963039,265.868016,14374.536053,-2493.708390,-1119.311648,0.026103,19.144775,-92.223586,-92.230828,19.141722,15,Q,580888.442737,2.116687e+06,581648.525054,2.117028e+06,Vertical


In [26]:
exportPath = 'C:/Users/BpAmos/Desktop/TMP/abkatuneWells.csv'
df_final.to_csv(os.path.join(exportPath),index=False)

In [33]:
appended_df.reset_index(drop=True,inplace = True)
appended_df

,MD,X,Y,Z,TVD,DX,DY,AZIM,INCL,DLS,wellId
0,0.0,587120.00,2133594.00,30.00,0.00,0.0000000000,0.00,0.000000,0.000000,0.0,Abkatun0001a
1,3603.0,587120.00,2133594.00,-3573.00,3603.00,0.0000000000,0.00,0.000000,0.000000,0.0,Abkatun0001a
2,0.0,587117.00,2133591.00,30.00,0.00,0.0000000000,0.00,0.000000,0.000000,0.0,Abkatun0001b
3,3384.0,587117.00,2133591.00,-3354.00,3384.00,0.0000000000,0.00,0.000000,0.000000,0.0,Abkatun0001b
4,0.0,585593.00,2131138.00,26.00,0.00,0.0000000000,0.00,136.847610,0.125659,0.0,Abkatun0004
...,...,...,...,...,...,...,...,...,...,...,...
10987,4360.0,581682.10,2116903.63,-4204.33,4234.33,-727.9000000,-337.37,266.000770,12.000263,0.0,Abkatun2141
10988,4410.0,581671.73,2116902.91,-4253.24,4283.24,-738.2700000,-338.09,266.002698,12.004661,0.0,Abkatun2141
10989,4460.0,581661.35,2116902.18,-4302.15,4332.15,-748.6500000,-338.82,266.002703,12.005852,0.0,Abkatun2141
10990,4510.0,581650.98,2116901.46,-4351.05,4381.05,-759.0200000,-339.54,265.868016,11.963039,0.0,Abkatun2141


In [36]:
df_final.reset_index(drop=True,inplace = True)
df_final

,wellId,md,inc,azim,tvd,e_w_deviation,n_s_deviation,dls,surface_latitude,surface_longitude,longitude_points,latitude_points,zone_number,zone_letter,x_points,y_points,surface_x,surface_y,isHorizontal
0,Abkatun0001a,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.295399,-92.170817,-92.170817,19.295399,15,Q,587118.698339,2.133722e+06,587118.698339,2.133722e+06,Vertical
1,Abkatun0001a,11820.86652,0.000000,0.000000,11820.866520,0.000000,0.000000,0.000000,19.295399,-92.170817,-92.170817,19.295399,15,Q,587118.698339,2.133722e+06,587118.698339,2.133722e+06,Vertical
2,Abkatun0001b,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.295372,-92.170845,-92.170845,19.295372,15,Q,587115.698385,2.133719e+06,587115.698385,2.133719e+06,Vertical
3,Abkatun0001b,11102.36256,0.000000,0.000000,11102.362560,0.000000,0.000000,0.000000,19.295372,-92.170845,-92.170845,19.295372,15,Q,587115.698385,2.133719e+06,587115.698385,2.133719e+06,Vertical
4,Abkatun0004,0.00000,0.125659,136.847610,0.000000,0.000000,0.000000,0.000000,19.273661,-92.185968,-92.185967,19.273661,15,Q,585538.062715,2.131308e+06,585538.062715,2.131308e+06,Vertical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10987,Abkatun2141,14304.46240,12.000263,266.000770,13893.159542,-2391.663258,-1112.140355,0.000000,19.144775,-92.223586,-92.230532,19.141741,15,Q,580919.546093,2.116689e+06,581648.525054,2.117028e+06,Vertical
10988,Abkatun2141,14468.50440,12.004661,266.002698,14053.615361,-2425.693395,-1114.518871,0.002681,19.144775,-92.223586,-92.230630,19.141735,15,Q,580909.173707,2.116688e+06,581648.525054,2.117028e+06,Vertical
10989,Abkatun2141,14632.54640,12.005852,266.002703,14214.069517,-2459.731380,-1116.897359,0.000726,19.144775,-92.223586,-92.230729,19.141729,15,Q,580898.798929,2.116688e+06,581648.525054,2.117028e+06,Vertical
10990,Abkatun2141,14796.58840,11.963039,265.868016,14374.536053,-2493.708390,-1119.311648,0.026103,19.144775,-92.223586,-92.230828,19.141722,15,Q,580888.442737,2.116687e+06,581648.525054,2.117028e+06,Vertical


In [37]:
merge_2 = pd.merge(df_final, appended_df,  left_index=True, right_index=True)
merge_2

,wellId_x,md,inc,azim,tvd,e_w_deviation,n_s_deviation,dls,surface_latitude,surface_longitude,...,X,Y,Z,TVD,DX,DY,AZIM,INCL,DLS,wellId_y
0,Abkatun0001a,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.295399,-92.170817,...,587120.00,2133594.00,30.00,0.00,0.0000000000,0.00,0.000000,0.000000,0.0,Abkatun0001a
1,Abkatun0001a,11820.86652,0.000000,0.000000,11820.866520,0.000000,0.000000,0.000000,19.295399,-92.170817,...,587120.00,2133594.00,-3573.00,3603.00,0.0000000000,0.00,0.000000,0.000000,0.0,Abkatun0001a
2,Abkatun0001b,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.295372,-92.170845,...,587117.00,2133591.00,30.00,0.00,0.0000000000,0.00,0.000000,0.000000,0.0,Abkatun0001b
3,Abkatun0001b,11102.36256,0.000000,0.000000,11102.362560,0.000000,0.000000,0.000000,19.295372,-92.170845,...,587117.00,2133591.00,-3354.00,3384.00,0.0000000000,0.00,0.000000,0.000000,0.0,Abkatun0001b
4,Abkatun0004,0.00000,0.125659,136.847610,0.000000,0.000000,0.000000,0.000000,19.273661,-92.185968,...,585593.00,2131138.00,26.00,0.00,0.0000000000,0.00,136.847610,0.125659,0.0,Abkatun0004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10987,Abkatun2141,14304.46240,12.000263,266.000770,13893.159542,-2391.663258,-1112.140355,0.000000,19.144775,-92.223586,...,581682.10,2116903.63,-4204.33,4234.33,-727.9000000,-337.37,266.000770,12.000263,0.0,Abkatun2141
10988,Abkatun2141,14468.50440,12.004661,266.002698,14053.615361,-2425.693395,-1114.518871,0.002681,19.144775,-92.223586,...,581671.73,2116902.91,-4253.24,4283.24,-738.2700000,-338.09,266.002698,12.004661,0.0,Abkatun2141
10989,Abkatun2141,14632.54640,12.005852,266.002703,14214.069517,-2459.731380,-1116.897359,0.000726,19.144775,-92.223586,...,581661.35,2116902.18,-4302.15,4332.15,-748.6500000,-338.82,266.002703,12.005852,0.0,Abkatun2141
10990,Abkatun2141,14796.58840,11.963039,265.868016,14374.536053,-2493.708390,-1119.311648,0.026103,19.144775,-92.223586,...,581650.98,2116901.46,-4351.05,4381.05,-759.0200000,-339.54,265.868016,11.963039,0.0,Abkatun2141


In [38]:
exportPath = 'C:/Users/BpAmos/Desktop/TMP/abkatuneWells2.csv'
merge_2.to_csv(os.path.join(exportPath),index=False)

In [ ]:

# def dms2dd(s):
#     """
#     convert degrees minutes seconds and direction into decimal degrees
#
#     :param s: string "degrees, minutes, seconds, direction"
#     :return: float, decimal degrees
#     """
#     # example: s = """0°51'56.29"S"""
#     #degrees, minutes, seconds, direction = re.split('[°\'"]+', s)
#     direction, degrees, minutes, seconds = s.split()
#     dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
#     if direction in ('S','W'):
#         dd*= -1
#     return dd